### Trigger Execution Order

I asked a question on Twitter the other day asking whether if you had multiple triggers on a table whether they executed serially or all at once. I got a reply with the answer (Thanks Aaron!), but figured it would be a good thing to test out here also.

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">One at a time. You can control first and last but that&#39;s it - the middles will run in an arbitrary / non-deterministic order. If you have &gt; 3 I don&#39;t know that order is going to be your biggest problem. :-) <a href="https://twitter.com/hashtag/sqlhelp?src=hash&amp;ref_src=twsrc%5Etfw">#sqlhelp</a></p>&mdash; Aaron Bertrand (@AaronBertrand) <a href="https://twitter.com/AaronBertrand/status/1121436026956861445?ref_src=twsrc%5Etfw">April 25, 2019</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>



First we'll create a test database and two tables:

In [1]:
IF NOT EXISTS(SELECT 1 FROM sys.databases where name = 'TriggerHappy')
    CREATE DATABASE TriggerHappy

GO

USE TriggerHappy

CREATE TABLE TriggerTable (
    TriggerTableId INT IDENTITY(1,1) PRIMARY KEY,
    TestName VARCHAR(15)
)

CREATE TABLE TimeStampTable (
    TimeStampTableId INT IDENTITY(1,1) PRIMARY KEY,
    InsertedTestName VARCHAR(15), 
    TriggerName VARCHAR(15),
    TimeExecuted DATETIME2 DEFAULT GETDATE()
)

I then created three triggers on the TriggerTable, each waits for 2 seconds and then inserts into the `TimeStampTable`.

In [2]:
USE TriggerHappy
GO

CREATE OR ALTER TRIGGER TriggerOne 
ON dbo.TriggerTable
AFTER INSERT
AS
DECLARE @test VARCHAR(15)

WAITFOR DELAY '00:00:02'
INSERT INTO TimeStampTable (InsertedTestName, TriggerName)
SELECT testname, 'TriggerOne' from inserted

GO 

CREATE OR ALTER TRIGGER TriggerTwo 
ON dbo.TriggerTable
AFTER INSERT
AS
WAITFOR DELAY '00:00:02'
INSERT INTO TimeStampTable (InsertedTestName, TriggerName)
SELECT testname, 'TriggerTwo' from inserted

GO

CREATE OR ALTER TRIGGER TriggerThree
ON dbo.TriggerTable
AFTER INSERT
AS
WAITFOR DELAY '00:00:02'
INSERT INTO TimeStampTable (InsertedTestName, TriggerName)
SELECT testname, 'TriggerThree' from inserted

Let's now insert a row and see what happens, this will run for 6ish seconds. Spoiler alert this is one of the reasons a lot of triggers might be a bad idea.

In [3]:
INSERT INTO TriggerTable (TestName)
VALUES ('TestOne')

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:06.0229917

Looking at the results and the timestamps we can see the triggers executed one after another.

In [4]:
Select * 
FROM TimeStampTable

(3 rows affected)

Total execution time: 00:00:00.1754582

TimeStampTableId,InsertedTestName,TriggerName,TimeExecuted
1,TestOne,TriggerOne,2019-05-03 22:49:30.6800000
2,TestOne,TriggerTwo,2019-05-03 22:49:32.6833333
3,TestOne,TriggerThree,2019-05-03 22:49:34.6866667


In [5]:
-- Cleanup

USE MASTER
GO
DROP DATABASE TriggerHappy